# Validate Density Calculation

Validate that radial density distribution is uniform for randomly distributed points in a spherical confinement.

### Navigate to Root Directory

In [ ]:
# Built-in modules
import os
import sys

# Insert package root to system path
cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)

print("Directory containing the notebook:")
print(cwd)

In [ ]:
# Change working directory to package root
os.chdir(parent_dir)
print("Root Directory of Package: ")
print(os.getcwd())

### Import Modules

In [ ]:
# External modules
import numpy as np
import matplotlib.pyplot as plt

# Package modules
from chromo.polymers import Chromatin
import chromo.binders
from chromo.fields import UniformDensityField

### Specify Reader Proteins

In [ ]:
# Instantiate the HP1 reader protein, which is pre-defined in the `chromo.binders` module
null_reader = chromo.binders.get_by_name('null_reader')

# Create the binder collection
binders = chromo.binders.make_binder_collection([null_reader])

### Specify Confinement

In [ ]:
confine_type = "Spherical"
confine_radius = 900

### Specify Polymer

In [1]:
num_beads = 100000
bead_spacing = 16.5

chemical_mods = np.atleast_2d(np.zeros(num_beads, dtype=int)).T
states = chemical_mods.copy()

polymer = Chromatin.confined_uniform_random_walk(
    'Chr-1',
    num_beads,
    bead_spacing,
    states=states,
    confine_type=confine_type,
    confine_length=confine_radius,
    binder_names=np.array(['null_reader']),
    chemical_mods=chemical_mods,
    chemical_mod_names=np.array(['null_mod'])
)

NameError: name 'np' is not defined

### Specify Field

In [ ]:
n_buffer = 2
n_accessible = 63
n_bins_x = n_accessible + n_buffer
n_bins_y = n_bins_x
n_bins_z = n_bins_x

x_width = 2 * confine_radius * (1 + n_buffer / n_accessible)
y_width = x_width
z_width = x_width

udf = UniformDensityField(
    polymers = [polymer],
    binders = binders,
    x_width = x_width,
    nx = n_bins_x,
    y_width = y_width,
    ny = n_bins_y,
    z_width = z_width,
    nz = n_bins_z,
    confine_type = confine_type,
    confine_length = confine_radius,
    chi = 1,
    assume_fully_accessible = 1
)

### Analyze Radial Densities

In [ ]:
r = np.asarray(polymer.r)
radial_dists = np.linalg.norm(r, axis=1)
step_size = 30
bins = np.arange(step_size, confine_radius, step_size)
counts, bin_edges = np.histogram(radial_dists, bins=bins)
counts = counts.astype(float)
for i in range(len(bin_edges)-1):
    volume = 4/3 * np.pi * ((bin_edges[i+1]/1E3)**3 - (bin_edges[i]/1E3)**3)
    counts[i] /= volume

plt.figure(figsize=(5,4), dpi=300)
plt.hist(bin_edges[:-1], bin_edges, weights=counts, alpha=0.75, color="blue")
plt.xlabel(r"Radial Position ($nm$)")
plt.ylabel(r"Bead Density ($1/nm^3$)")
plt.show()